In [ ]:
!nvidia-smi

Sun Feb  6 05:02:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install --ignore-installed PyYAML
import yaml
print(yaml.__version__)

     |████████████████████████████████| 596 kB 12.5 MB/s 
6.0


In [ ]:
import os
import shutil
import json

from tqdm import tqdm
from glob import glob
from PIL import Image

import torch
import numpy as np
import random

In [ ]:
# Google Drive mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/ultralytics/yolov5 
!pip -q install -r /content/yolov5/requirements.txt
!pip -q install wandb

Cloning into 'yolov5'...
remote: Enumerating objects: 10878, done.
remote: Total 10878 (delta 0), reused 0 (delta 0), pack-reused 10878
Receiving objects: 100% (10878/10878), 10.95 MiB | 5.93 MiB/s, done.
Resolving deltas: 100% (7519/7519), done.
     |████████████████████████████████| 1.7 MB 15.6 MB/s 
     |████████████████████████████████| 180 kB 57.6 MB/s 
     |████████████████████████████████| 143 kB 66.1 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 


### Dataset 불러오기

In [ ]:
!tar -xvf /content/drive/MyDrive/AI_Factory_YOLO/Train_Dataset_Full.tar

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
content/s_data/Training/cow_label/livestock_cow_bbox_001557.json
content/s_data/Training/cow_label/livestock_cow_bbox_000595.json
content/s_data/Training/cow_label/livestock_cow_bbox_009379.json
content/s_data/Training/cow_label/livestock_cow_bbox_011250.json
content/s_data/Training/cow_label/livestock_cow_bbox_009601.json
content/s_data/Training/cow_label/livestock_cow_bbox_004481.json
content/s_data/Training/cow_label/livestock_cow_bbox_001120.json
content/s_data/Training/cow_label/livestock_cow_bbox_009734.json
content/s_data/Training/cow_label/livestock_cow_bbox_011786.json
content/s_data/Training/cow_label/livestock_cow_bbox_002592.json
content/s_data/Training/cow_label/livestock_cow_bbox_005087.json
content/s_data/Training/cow_label/livestock_cow_bbox_007918.json
content/s_data/Training/cow_label/livestock_cow_bbox_012827.json
content/s_data/Training/cow_label/livestock_cow_bbox_010591.json
content/s_data/Training/cow_label/livestock_cow_bbox_0

In [ ]:
# Train Image
if not os.path.exists('/content/images'):
    os.mkdir('/content/images')
if not os.path.exists('/content/images/train'):
    os.mkdir('/content/images/train')

source_cow = '/content/content/s_data/Training/cow_image/'
source_pig = '/content/content/s_data/Training/pig_image/'
destination = '/content/images/train'

cow = os.listdir(source_cow)
pig = os.listdir(source_pig)

for c in tqdm(cow):
    shutil.move(source_cow + c, destination)
for p in tqdm(pig):
    shutil.move(source_pig + p, destination)

100%|██████████| 4303/4303 [00:00<00:00, 24715.66it/s]


In [ ]:
# Train label
if not os.path.exists('/content/labels_json'):
    os.mkdir('/content/labels_json')
if not os.path.exists('/content/labels_json/train'):
    os.mkdir('/content/labels_json/train')

source_cow = '/content/content/s_data/Training/cow_label/'
source_pig = '/content/content/s_data/Training/pig_label/'
destination = '/content/labels_json/train'

cow = os.listdir(source_cow)
pig = os.listdir(source_pig)

for c in tqdm(cow):
    shutil.move(source_cow + c, destination)
for p in tqdm(pig):
    shutil.move(source_pig + p, destination)

100%|██████████| 4303/4303 [00:00<00:00, 26078.53it/s]


In [ ]:
# Valid Image & Label
if not os.path.exists('/content/images'):
    os.mkdir('/content/images')
if not os.path.exists('/content/images/valid'):
    os.mkdir('/content/images/valid')

source = '/content/images/train/'
destination = '/content/images/valid'
source_file = os.listdir(source)
source_file.sort()
source_file = source_file[0]
shutil.copy(source + source_file, destination)

if not os.path.exists('/content/labels_json'):
    os.mkdir('/content/labels_json')
if not os.path.exists('/content/labels_json/valid'):
    os.mkdir('/content/labels_json/valid')

source = '/content/labels_json/train/'
destination = '/content/labels_json/valid'
source_file = os.listdir(source)
source_file.sort()
source_file = source_file[0]
shutil.copy(source + source_file, destination)

'/content/labels_json/valid/livestock_cow_bbox_000001.json'

In [ ]:
train_image = glob('/content/images/train/*.jpg')
train_label = glob('/content/labels_json/train/*.json')

valid_image = glob('/content/images/valid/*.jpg')
valid_label = glob('/content/labels_json/valid/*.json')

test_image = glob('/content/images/test/*.jpg')
test_label = glob('/content/labels_json/test/*.json')

In [ ]:
print('Train Data의 개수:', len(train_image))
print('Train Data의 개수:', len(train_label))

print('Valid Data의 개수:', len(valid_image))
print('Valid Data의 개수:', len(valid_label))

print('Test Data의 개수', len(test_image))
print('Test Data의 개수', len(test_label))

Train Data의 개수: 16455
Train Data의 개수: 16455
Valid Data의 개수: 1
Valid Data의 개수: 1
Test Data의 개수 0
Test Data의 개수 0


### YOLOv5 Setting에 맞도록 Label 변경

In [ ]:
# x1, y1, x2, y2 -> x, y , w, h로 변경
# 이미지 크기에 맞게 정규화
if not os.path.exists('/content/labels'):
    os.mkdir('/content/labels')
if not os.path.exists('/content/labels/test'):
    os.mkdir('/content/labels/test')
if not os.path.exists('/content/labels/train'):
    os.mkdir('/content/labels/train')
if not os.path.exists('/content/labels/valid'):
    os.mkdir('/content/labels/valid')

train_label = glob('/content/labels_json/train/*.json')
valid_label = glob('/content/labels_json/valid/*.json')
train_label.sort()
valid_label.sort()

In [ ]:
def convert_coordinate(width, height, x1, y1, x2, y2):
    dw = 1./ width
    dh = 1./ height
    x = (x1 + x2) / 2.0
    y = (y1 + y2) / 2.0
    w = x2 - x1
    h = y2 - y1
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x, y, w, h)

In [ ]:
# Train Label 변경
for label in tqdm(train_label):
    with open(label, 'r') as f:
        json_data = json.load(f)
    
    height = json_data['label_info']['image']['height']
    width = json_data['label_info']['image']['width']
    box_info = json_data['label_info']['annotations']
    
    image_name = label.replace('labels_json', 'labels')
    image_name = image_name.replace('json', 'txt')
    
    with open(image_name, 'w') as f:
        for i in range(len(box_info)):
            x1, y1, x2, y2 = box_info[i]['bbox']
            if 'cow' in image_name:
                category = 1
            else:
                category = 0
            x, y, w, h = convert_coordinate(width, height, x1, y1, x2, y2)
            data = str(category) + ' ' + str(x) + ' ' + str(y) +  ' ' + str(w) + ' ' + str(h) + '\n'
            f.write(data)

100%|██████████| 16455/16455 [00:02<00:00, 6244.79it/s]


In [ ]:
# Valid Label 변경
for label in tqdm(valid_label):
    with open(label, 'r') as f:
        json_data = json.load(f)
    
    height = json_data['label_info']['image']['height']
    width = json_data['label_info']['image']['width']
    box_info = json_data['label_info']['annotations']
    
    image_name = label.replace('labels_json', 'labels')
    image_name = image_name.replace('json', 'txt')
    
    with open(image_name, 'w') as f:
        for i in range(len(box_info)):
            x1, y1, x2, y2 = box_info[i]['bbox']
            if 'cow' in image_name:
                category = 1
            else:
                category = 0
            x, y, w, h = convert_coordinate(width, height, x1, y1, x2, y2)
            data = str(category) + ' ' + str(x) + ' ' + str(y) +  ' ' + str(w) + ' ' + str(h) + '\n'
            f.write(data)

100%|██████████| 1/1 [00:00<00:00, 1158.01it/s]


In [ ]:
train_label = glob('/content/labels/train/*.txt')
valid_label = glob('/content/labels/valid/*.txt')

print('Train Data의 개수:', len(train_label))
print('Valid Data의 개수:', len(valid_label))

Train Data의 개수: 16455
Valid Data의 개수: 1


In [ ]:
# 리스트 이름을 txt파일로 생성
train_list = glob('/content/images/train/*.jpg')
valid_list = glob('/content/images/valid/*.jpg')
test_list = glob('/content/images/test/*.jpg')
 
with open('/content/train.txt', 'w') as f:
	f.write('\n'.join(train_list) + '\n')
    
with open('/content/valid.txt', 'w') as f:
	f.write('\n'.join(valid_list) + '\n')
 
with open('/content/test.txt', 'w') as f:
  f.write('\n'.join(test_list) + '\n')

In [ ]:
# coco.yaml을 바탕으로 우리 데이터셋 이름대로 변경하여 data.yaml로 저장
with open('/content/yolov5/data/coco.yaml', 'r') as f:
	data = yaml.load(f, Loader=yaml.FullLoader)
     
data['path'] = '/content' 
data['train'] = 'train.txt'
data['val'] = 'valid.txt'
data['test'] = 'test.txt'
data['nc'] = 2
data['names'] = ['pig','cow']
 
with open('/content/data.yaml', 'w') as f:
	yaml.dump(data, f)

### 학습 진행

In [ ]:
!python /content/yolov5/train.py --batch 10 --epochs 20 --data '/content/data.yaml' --cfg '/content/yolov5/models/hub/yolov5m6.yaml' --weights yolov5m6.pt --img 1280 --name m6_pretrained_full_b10_e20_hyp_tuning_v1_linear --linear-lr --hyp '/content/drive/MyDrive/AI_Factory/hyp/experiment_hyp_v1.yaml'

In [ ]:
import shutil

source = '/content/yolov5/runs/train/m6_pretrained_full_b10_e20_hyp_tuning_v1_linear/weights/best.pt'
destination = '/content/drive/MyDrive/AI_Factory/trained_model/m6_pretrained_full_b10_e20_hyp_tuning_v1_linear.pt'
shutil.move(source,destination)

source = '/content/yolov5/runs/train/m6_pretrained_full_b10_e20_hyp_tuning_v1_linear/hyp.yaml'
destination = '/content/drive/MyDrive/AI_Factory/trained_model/hyp_m6_pretrained_full_b10_e20_hyp_tuning_v1_linear.yaml'
shutil.move(source,destination)

source = '/content/yolov5/runs/train/m6_pretrained_full_b10_e20_hyp_tuning_v1_linear/opt.yaml'
destination = '/content/drive/MyDrive/AI_Factory/trained_model/opt_m6_pretrained_full_b10_e20_hyp_tuning_v1_linear.yaml'
shutil.move(source,destination)